In [ ]:
# get text and format it    


with open('<file_path>.txt', encoding="latin-1") as f:
    transcript = f.readlines()
    transcript

In [ ]:
# format in json to feed azure api

import re
import json



def parse_vtt(transcript):
    entries = []
    segments = re.split(r"\d+:\d+:\d+\.\d+ --> \d+:\d+:\d+\.\d+\n", transcript.strip())
    for idx, segment in enumerate(segments[1:], start=1):  # Skip the first split, which is empty
        lines = segment.strip().split('\n')
        if len(lines) >= 2:
            speaker, text = lines[0], lines[1]
            role = speaker 
            entries.append({
                "text": text,
                "id": str(idx),
                "role": role,
                "participantId": speaker.replace(" ", "_"),
            })
    return entries

conversation_items = parse_vtt("".join(transcript))
conversation_json = {
    "conversations": [
        {
            "conversationItems": conversation_items,
            "modality": "text",
            "id": "conversation1",
            "language": "fr",
        }
    ]
}

print(json.dumps(conversation_json, indent=2, ensure_ascii=False))


In [ ]:
import os

key = 
endpoint = 

from azure.core.credentials import AzureKeyCredential
from azure.ai.language.conversations import ConversationAnalysisClient

client = ConversationAnalysisClient(endpoint, AzureKeyCredential(key))
with client:
    poller = client.begin_conversation_analysis(
        task={
            "displayName": "Analyze conversations from xxx",
            "analysisInput": {
                "conversations": conversation_json['conversations']
            },
            "tasks": [
                {
                    "taskName": "Issue task",
                    "kind": "ConversationalSummarizationTask",
                    "parameters": {"summaryAspects": ["issue"]},
                },
                {
                    "taskName": "Resolution task",
                    "kind": "ConversationalSummarizationTask",
                    "parameters": {"summaryAspects": ["resolution"]},
                },
            ],
        }
    )

    # view result
    result = poller.result()
    task_results = result["tasks"]["items"]
    for task in task_results:
        print(f"\n{task['taskName']} status: {task['status']}")
        task_result = task["results"]
        if task_result["errors"]:
            print("... errors occurred ...")
            for error in task_result["errors"]:
                print(error)
        else:
            conversation_result = task_result["conversations"][0]
            if conversation_result["warnings"]:
                print("... view warnings ...")
                for warning in conversation_result["warnings"]:
                    print(warning)
            else:
                summaries = conversation_result["summaries"]
                for summary in summaries:
                    print(f"{summary['aspect']}: {summary['text']}")